<a href="https://colab.research.google.com/github/ethmanemahmoudethmane/classifieur-d-motions/blob/main/classifieur_d%E2%80%99%C3%A9motions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install "numpy<2.0"

In [4]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [ ]:
!pip install transformers torch pandas numpy datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# 1. Charger le dataset
df = pd.read_parquet("/content/train-00000-of-00001.parquet")

# Mapper les labels numériques aux émotions
label_to_emotion = {0: "sadness", 1: "joy", 2: "anger", 3: "fear", 4: "surprise", 5: "neutral"}
df["emotion"] = df["label"].map(label_to_emotion)

# Afficher la distribution des émotions
print(df["label"].value_counts())

# 2. Préparer les données
# Convertir le DataFrame en dataset Hugging Face
dataset = Dataset.from_pandas(df[["text", "label"]])

# Diviser en train et validation
train_val_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]

# 3. Tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Appliquer la tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Configurer les colonnes pour l'entraînement
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 4. Charger le modèle
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_to_emotion)
)
# 5. Définir les métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "f1": f1}



label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/333447 [00:00<?, ? examples/s]

Map:   0%|          | 0/83362 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    eval_steps=200,
    learning_rate=3e-5,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=80,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 7. Entraîner le modèle
trainer.train()

# 8. Sauvegarder le modèle
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

# 9. Prédiction sur un exemple
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=-1).item()
    return label_to_emotion[predicted_class]

# Exemple de prédiction
sample_text = "I'm so happy today!"
print(f"Emotion prédite : {predict_emotion(sample_text)}")

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.081400,0.088988,0.942096,0.942984
2,0.081800,0.088690,0.942060,0.943464
3,0.080400,0.093037,0.939601,0.940194


Emotion prédite : joy


In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.08868967741727829, 'eval_accuracy': 0.942059931383604, 'eval_f1': 0.9434637082772691, 'eval_runtime': 62.417, 'eval_samples_per_second': 1335.565, 'eval_steps_per_second': 16.71, 'epoch': 3.0}


In [ ]:
sample_text = "I miss her so much, everything feels empty now."
print(f"Emotion prédite : {predict_emotion(sample_text)}")

Emotion prédite : sadness


In [ ]:

sample_text = "I just got accepted into my dream university!"
print(f"Emotion prédite : {predict_emotion(sample_text)}")

Emotion prédite : joy


In [ ]:
sample_text =
print(f"Emotion prédite : {predict_emotion(sample_text)}")

Emotion prédite : joy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(df[["label", "emotion", "text"]].sample(10))

        label   emotion                                               text
36130       4  surprise      i just feel really helpless and heavy hearted
138065      0   sadness  ive enjoyed being able to slouch about relax a...
146440      4  surprise  i gave up my internship with the dmrg and am f...
103337      0   sadness                         i dont know i feel so lost
315528      4  surprise  i am a kindergarten teacher and i am thoroughl...
414303      0   sadness         i was beginning to feel quite disheartened
407867      2     anger  i would think that whomever would be lucky eno...
291389      1       joy  i fear that they won t ever feel that deliciou...
227948      5   neutral  im forever taking some time out to have a lie ...
176638      0   sadness  i can still lose the weight without feeling de...
